In [ ]:
!pip install dataset
import requests
import dataset
import re
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

In [ ]:
db = dataset.connect('sqlite:///books.db')

base_url = 'http://books.toscrape.com/'

def scrape_books(html_soup, url):
        books = html_soup.select('article.product_pod')
        for book in books:
                #とりあえず書籍のURLを保存する
                book_url = book.find('h3').find('a').get('href')
                book_url = urljoin(url, book_url)
                path = urlparse(book_url).path #path属性を取得する
                book_id = path.split('/')[1]
                #upsertはまず更新を試行、その後挿入を実行
                db['nooks'].upsert({
                    'book_id': book_id, 'last_seen': datetime.now() 
                }, ['book_id'])

def scrape_book(html_soup, book_id):
        main = html_soup.find(class_ = 'product_main')
        book = {}
        book['book_id'] = book_id
        book['title'] = main.find('h1').get_text(strip = True)
        book['price'] = main.select('#content_inner > article > div.row > div.col-sm-6.product_main > p.price_color')[0].text
        book['stock'] = main.find(class_ = 'instock_availability').get_text(strip = True)
        book['rating'] = ' '.join(
            main.find(class_ = 'star-rating').get('class').replace('star-rating', '').strip())
        book['img'] = html_soup.select('#product_gallery > div > div > div > img').get('src')
        book['description'] = html_soup.select('#content_inner > article > p')[0].text
        
        book_product_table = html_soup.find(text = 'Product Information').find_next('table')
        for row in book_product_table.find_all('tr'):
                header = row.find('th').get_text(strip = True)
                #ヘッダーを綺麗にする（？）
                header = re.sub('[^a-zA-Z]+', '_', header)
                value = row.find('td').get_text(strip = True)
                book[header] = value
        db['book_info'].upsert(book, ['book_id'])

#カタログ内のページをスクレイピング
url = base_url
inp = input('Do you wish to re-shape the catalogue (y/n)?')
while True and inp == 'y':
        print('Now...', url)
        res = requests.get(url)
        html_soup = BeautifulSoup(res.text, 'html.parser')
        scrape_books(html_soup, url)
        #次のページはある？
        next_a = html_soup.select('#default > div > div > div > div > section > div:nth-child(2) > div > ul > li.next')
        if next_a is None or next_a[0].get('href') is not None:
                break
        url = urljoin(url, next_a[0].get('href'))

#古いものから順に、書籍ごとにスクレイピング
books = db['books'].find(order_by = ['lst_seen'])
for book in books:
        book_id = book['book_id']
        book_url = f'{base_url}catalog/'
        print('Now...', book_url)
        res = requests.get(url)
        res.encoding = 'utf-8'
        html_soup = BeautifulSoup(res.text, 'html.parser')
        scrape_book(html_soup, book_id)
        #最後に見たタイムスタンプを更新する
        db['books'].upsert({
            'book_id': book_id, 'last_seen': datetime.now()},
             ['book_id'])
        
        

Do you wish to re-shape the catalogue (y/n)?y
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
Now... http://books.toscrape.com/
No

In [34]:
db = dataset.connect('sqlite:///books.db')

url = 'http://books.toscrape.com/'
res = requests.get(url)
html_soup = BeautifulSoup(res.text, 'html.parser')
books = html_soup.select('article.product_pod')
book = books[0]
#default > div > div > div > div > section > div:nth-child(2) > ol > li:nth-child(1) > article
print(book.find('h3').find('a').get('href'))
print(book.find('h3').find('a'))
print(book.find('h3'))
print(book.select('h3 > a')) #Resultsetで返されるのはイテラブルなので、インデックスで指定して取り出す必要がある
print(book.select('h3 > a')[0].get('href'))

catalogue/a-light-in-the-attic_1000/index.html
<a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
[<a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>]
catalogue/a-light-in-the-attic_1000/index.html


In [72]:
#content_inner > article
#content_inner > article > div.row > div.col-sm-6.product_main > h1
url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
res = requests.get(url)
html_soup = BeautifulSoup(res.text, 'html.parser')
main = html_soup.select('#content_inner > article > div.row > div.col-sm-6.product_main > p.price_color')
print(main[0].text)

Â£51.77


In [116]:
book= html_soup.select('#content_inner > article > table')
book[0].get('th')